This notebook came from the frip notebook, 

# transcriptomic binding qc

In [ ]:
results = []

for name, row in merged_data.iterrows():
    path, ext = os.path.splitext(os.path.basename(row.CLIP_rep1))
    rep1_metrics = path + ".in_genes.metrics"

    results.append(format_frip_analysis(row.CLIP_rep1, "/home/gpratt/clipper/clipper/data/regions/hg19_genes.bed", rep1_metrics))
    
    path, ext = os.path.splitext(os.path.basename(row.CLIP_rep2))
    rep2_metrics = path + ".in_genes.metrics"
    
    results.append(format_frip_analysis(row.CLIP_rep2, "/home/gpratt/clipper/clipper/data/regions/hg19_genes.bed", rep2_metrics))

In [ ]:
job_maker.make_script(results,
                     "/home/gpratt/projects/encode/scripts/genic_binding_v1",
                     "/home/gpratt/projects/encode/analysis/genic_binding/",
                     walltime=1)

In [ ]:
# intronic vs exonic binders

In [ ]:
results = []

for name, row in merged_data.iterrows():
    path, ext = os.path.splitext(os.path.basename(row.CLIP_rep1))
    rep1_metrics = path + ".in_genes.metrics"

    results.append(format_frip_analysis(row.CLIP_rep1, "/home/gpratt/clipper/clipper/data/regions/hg19_exons.bed", rep1_metrics))
    
    path, ext = os.path.splitext(os.path.basename(row.CLIP_rep2))
    rep2_metrics = path + ".in_genes.metrics"
    
    results.append(format_frip_analysis(row.CLIP_rep2, "/home/gpratt/clipper/clipper/data/regions/hg19_exons.bed", rep2_metrics))
    
    path, ext = os.path.splitext(os.path.basename(row.INPUT))
    rep1_metrics = path + ".in_genes.metrics"

    results.append(format_frip_analysis(row.INPUT, "/home/gpratt/clipper/clipper/data/regions/hg19_exons.bed", rep1_metrics))


In [ ]:
job_maker.make_script(results,
                     "/home/gpratt/projects/encode/scripts/exon_binding_v1",
                     "/home/gpratt/projects/encode/analysis/exon_binding/",
                     walltime=1)

In [ ]:
# Define broad vs narrow binders?

In [ ]:
def get_mean_merged_length(fn):
    bedtool = pybedtools.BedTool(fn)
    bedtool = bedtool.sort().merge(s=True, d=100)
    return np.mean([len(interval) for interval in bedtool])

In [ ]:
merged_data['rep1_lengths'] = merged_data['uID_01.basedon_uID_01.peaks.l2inputnormnew.bed.compressed.bed'].apply(get_mean_merged_length)

In [ ]:
sns.distplot(merged_data['rep1_lengths'])

In [ ]:
merged_data[merged_data.RBP == "TAF15"].rep2_filtered_lineant[80]

In [ ]:
merged_data['rep2_lengths'] = merged_data['uID_02.basedon_uID_02.peaks.l2inputnormnew.bed.compressed.bed'].apply(get_mean_merged_length)

In [ ]:
results = []

for name, row in merged_data.iterrows():

    path, ext = os.path.splitext(os.path.basename(row.CLIP_rep2))
    rep2_metrics = path + ".in_genes.metrics"
    
    results.append(format_frip_analysis(row.CLIP_rep2, "/home/gpratt/clipper/clipper/data/regions/hg19_genes.bed", rep2_metrics))

In [ ]:
merged_data['rep1_neg_bw'] = merged_data.CLIP_rep1.apply(lambda x: os.path.splitext(x)[0] + ".norm.neg.bw")
merged_data['rep1_pos_bw'] = merged_data.CLIP_rep1.apply(lambda x: os.path.splitext(x)[0] + ".norm.pos.bw")

In [ ]:
rbp_map = splicing_map.ReadDensity(merged_data['rep1_neg_bw'][1], merged_data['rep1_pos_bw'][1])
peaks_of_interest = pybedtools.BedTool(merged_data['uID_01.basedon_uID_01.peaks.l2inputnormnew.bed.compressed.bed'][1])

In [ ]:
only_bound_regions = foo.intersect(bar, u=True, s=True)

In [ ]:
gene_coverage = bamtool.coverage(only_bound_regions, s=True)

In [ ]:
gene_coverage.head()

Basic idea, get regions that are already thought to be bound by peaks, look to see if reads in those regions are broadly distributed or narrowly distributed

In [ ]:
len(only_bound_regions)

In [ ]:
result = []
for interval in only_bound_regions:
    baz = rbp_map.values(interval.chrom, interval.start, interval.stop, interval.strand)
    result.append(np.median(np.array(baz) / sum(baz)))

In [ ]:
# Trying just very basic coverage test

In [ ]:
merged_data[merged_data.RBP == "HNRNPA2B1"]

In [ ]:
hg19_genes = pybedtools.BedTool("/home/gpratt/clipper/clipper/data/regions/hg19_genes.bed").sort().saveas()

In [ ]:
def get_expressed_gene_coverage(peaks, bam_file):
    peaks_of_interest = pybedtools.BedTool(peaks).sort()
    bamtool = pybedtools.BedTool(bam_file)
    only_bound_regions = hg19_genes.intersect(peaks_of_interest, u=True, s=True, sorted=True)
    gene_coverage = bamtool.coverage(only_bound_regions, s=True)
    coverage = [float(interval[-1]) for interval in gene_coverage]
    return coverage

In [ ]:
taf15_coverage = get_expressed_gene_coverage(merged_data['uID_01.basedon_uID_01.peaks.l2inputnormnew.bed.compressed.bed'][250], 
                                             merged_data.CLIP_rep1[250])

In [ ]:
gene_coverage = {}
for name, row in merged_data.iterrows():
    gene_coverage[row.uID] = get_expressed_gene_coverage(row['uID_01.basedon_uID_01.peaks.l2inputnormnew.bed.compressed.bed'], 
                                row.CLIP_rep1)

In [ ]:
num_rows = 1 
num_cols = 1 

results = {}
with dataviz.Figure(os.path.join(img_dir, "foo.svg"), figsize=(10 * num_cols, 10*num_rows)) as fig:
    ax = fig.add_subplot(1,1,1)
    
    for name, row in gene_coverage.iteritems():
        dataviz.plot_pdf(row, label=name, ax=ax)
        pdf = np.concatenate(dataviz.pdf(row)[1])
        results[merged_data[merged_data.uID == name].RBP.values[0]] = pdf
        
    ax.set_xlim(0,1)

In [ ]:
foo = pd.DataFrame(results).T

In [ ]:
img_dir = "/home/gpratt/Dropbox/Pratt,Gabriel/PapersInProgress/eCLIP_qc/working_figures/fig_2"

In [ ]:
cg = sns.clustermap(foo, col_cluster=False, figsize=(10,40))
n = plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
fig = plt.gcf()
fig.savefig(os.path.join(img_dir, "distributions.svg"))

In [ ]:
# the general coverage kind of works
It fails for a bunch of narrow binders DGRC8 is one great example, RAVER1 is known to point source repress splicing, QKI should also be point source.  Switching to looking for enrichment over background

In [ ]:
hg19_genes = pybedtools.BedTool("/home/gpratt/clipper/clipper/data/regions/hg19_genes.bed")
gene_lengths = {interval.name: len(interval) for interval in hg19_genes}
gene_lengths = pd.Series(gene_lengths)
hg19_windows = pybedtools.BedTool("/home/gpratt/projects/encode/analysis/ad-hoc/windowing_enrichment/hg19_windows.bed")

In [ ]:
def fold_enrichment_coverage(ip_file, input_file, hg19_windows):
    ip_bamtool = pybedtools.BedTool(ip_file)
    input_bamtool = pybedtools.BedTool(input_file)
    
    ip_tool = ip_bamtool.coverage(hg19_windows, counts=True)
    input_tool = input_bamtool.coverage(hg19_windows, counts=True)
    
    ip_mapped_reads = pysam.Samfile(ip_file).mapped
    input_mapped_reads = pysam.Samfile(input_file).mapped

    ip_mapped_reads /= 1000000.
    input_mapped_reads /= 1000000.
    
    ip_df = ip_tool.to_dataframe()
    input_df = input_tool.to_dataframe()
    
    ip_df.score = (ip_df.score + 1) / ip_mapped_reads
    input_df.score = (input_df.score + 1) / input_mapped_reads
    ip_df['region_length'] = ip_df.end - ip_df.start
    ip_df['input_score'] = input_df.score
    
    ip_df['log2_fold_enrichment'] = np.log2(ip_df.score  / input_df.score)
    return ip_df

In [ ]:
ip_df_dgcr8 = fold_enrichment_coverage("/projects/ps-yeolab3/encode/analysis/encode_master/341_01_DGCR8.merged.r2.bam",
                         "/projects/ps-yeolab3/encode/analysis/encode_master/341_INPUT_TCCGGAGA-CCTATCCT_L002_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam",
                            hg19_windows
                        )

In [ ]:
filtered_df = ip_df_dgcr8[ip_df_dgcr8.log2_fold_enrichment > 4]
covered_regions = filtered_df.groupby("name").region_length.sum()
sns.distplot((covered_regions / gene_lengths).dropna())

In [ ]:
filtered_df.region_length.sum()

In [ ]:
ip_df_rbfox2 = fold_enrichment_coverage("/projects/ps-yeolab3/encode/analysis/encode_v12/204_01_RBFOX2.merged.r2.bam",
                         "/projects/ps-yeolab2/encode/analysis/encode_v12/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam",
                            hg19_windows
                        )

In [ ]:
filtered_df = ip_df_rbfox2[ip_df_rbfox2.log2_fold_enrichment > 4]
covered_regions = filtered_df.groupby("name").region_length.sum()
sns.distplot((covered_regions / gene_lengths).dropna())

In [ ]:
merged_data[merged_data.RBP == "TAF15"]

In [ ]:
gene_coverage_enriched = {}
for name, row in merged_data.iterrows():
    ip_df = fold_enrichment_coverage(row.CLIP_rep1, row.INPUT, hg19_windows)
    
    filtered_df = ip_df[ip_df.log2_fold_enrichment > 4]
    covered_regions = filtered_df.groupby("name").region_length.sum()
    gene_coverage_enriched[row.uID] = (covered_regions / gene_lengths).dropna()

In [ ]:
row = merged_data[merged_data.RBP == "TAF15"].ix[80]

In [ ]:
ip_bamtool = pybedtools.BedTool(row.CLIP_rep1)

In [ ]:
ip_tool = ip_bamtool.coverage(hg19_windows, counts=True)


In [ ]:
ip_tool.to_dataframe()

In [ ]:
ip_df = fold_enrichment_coverage(row.CLIP_rep1, row.INPUT, hg19_windows)
filtered_df = ip_df[ip_df.log2_fold_enrichment > 4]


In [ ]:
ip_df[ip_df.log2_fold_enrichment > 2]

In [ ]:
filtered_df

In [ ]:
num_rows = 1 
num_cols = 1 

results = {}
mean_results = {}
with dataviz.Figure(os.path.join(img_dir, "foo.svg"), figsize=(10 * num_cols, 10*num_rows)) as fig:
    ax = fig.add_subplot(1,1,1)
    
    for name, row in gene_coverage_enriched.iteritems():
        if len(row) == 0:
            continue
            
        dataviz.plot_pdf(row, label=name, ax=ax)
        pdf = np.concatenate(dataviz.pdf(row)[1])
        rbp_name = merged_data[merged_data.uID == name].RBP.values[0] + "_" + merged_data[merged_data.uID == name]['Cell line'].values[0]
        results[rbp_name] = pdf
        mean_results[rbp_name] = np.mean(row)
        
    ax.set_xlim(0,1)

In [ ]:
foo = pd.DataFrame(results).T

In [ ]:
cg = sns.clustermap(foo, col_cluster=False, figsize=(10,40))
n = plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
fig = plt.gcf()
fig.savefig(os.path.join(img_dir, "distributions.svg"))

In [ ]:
mean_results = pd.Series(mean_results)

In [ ]:
mean_results.sort_values()

# exonic vs intronic

In [ ]:
%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
import numpy as np
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns
import matplotlib
from oauth2client.client import SignedJwtAssertionCredentials
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter


img_dir = "/home/gpratt/Dropbox/encode_integration/qc_work/"

In [ ]:
json_key = json.load(open("../public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest['qc_id'] = manifest.apply(lambda x: "{}_{}".format(x.ENCODE_ID, x.RBP), axis=1)
manifest.is_encode = manifest.is_encode == "TRUE"

In [ ]:
def add_metrics_v2(metrics_files, name):
    filtered_frip_v12 = pd.concat({"_".join(os.path.basename(metrics_file).split("_")[:2]) : pd.read_table(metrics_file) for metrics_file in metrics_files})

    return encode_only_qc

In [ ]:
add_metrics_v2(glob.glob("/home/gpratt/projects/encode/analysis/exon_binding/*.metrics"),
               "exonic_binding")